In [2]:
def set_ch():
    from pylab import mpl
    mpl.rcParams['font.sans-serif'] = ['FangSong'] # 指定默认字体
    mpl.rcParams['axes.unicode_minus'] = False # 解决保存图像是负号'-'显示为方块的问题
set_ch()
import matplotlib.pyplot as plt
import pandas as pd

pd.set_option('display.notebook_repr_html',True)
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',150)
pd.set_option('display.max_seq_items',None)
%matplotlib inline

In [3]:
# -*- coding: utf-8 -*-
def test_model(dfs, results, names):#返回模型误差，R²,每个用户最好的模型
    import numpy as np
    import pandas as pd
    import math
    from sklearn import cross_validation
    from sklearn.linear_model import ElasticNet
    from sklearn.tree import DecisionTreeRegressor
    from sklearn.linear_model import Ridge
    from sklearn.linear_model import Lasso
    from sklearn.linear_model import LinearRegression
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.ensemble import GradientBoostingRegressor
    from sklearn.ensemble import ExtraTreesRegressor
    from sklearn import cross_validation, metrics
    from statsmodels.tsa.arima_model import ARIMA
    from xgboost import XGBRegressor
    from sklearn.ensemble import AdaBoostRegressor
    err_df = pd.DataFrame(index=['Linear_','Ridge_','Lasso_','ElasticNet_','CART_err_','RForest_','GBRT_','ExtraTree_','XGB_','Ada_'])
    corrcoef_df = pd.DataFrame(index=['Linear_','Ridge_','Lasso_','ElasticNet_','CART_err_','RForest_','GBRT_','ExtraTree_','XGB_','Ada_'])
    
    models = []
    best_models = {}
    best_models_name = {}
    best_errdf = {}
    models.append(('Linear_err', LinearRegression()))
    models.append(('Ridge_err', Ridge()))
    models.append(('Lasso_err', Lasso()))
    models.append(('EticNet_err', ElasticNet()))
    models.append(('CART_err', DecisionTreeRegressor()))
    models.append(('RF_err', RandomForestRegressor()))
    models.append(('GBRT_err', GradientBoostingRegressor()))
    models.append(('Extrtree_err', ExtraTreesRegressor()))
    models.append(('Xgboost_err', XGBRegressor()))
    models.append(('Ada_err', AdaBoostRegressor()))

    for rr in names:
        user_dfs = dfs[rr]
        
        y_prediction = []
        X_train = user_dfs[0:train_num]
        Y_train = results[rr][0:train_num]

        X_test = user_dfs[train_num:train_test_num]
        Y_test = results[rr][train_num:train_test_num]
        best_err = 1000000000

        err = []
        corrcoef = []
        #print rr,'\t\t','corrcoef_df\t','err'
        for name, model in models:
            model.fit(X_train, Y_train)
            Y_pred = model.predict(X_test)

            sum_  =  ( (Y_test - Y_pred) /(Y_test)) *  (  ((Y_test - Y_pred) ) / (Y_test) )#( abs((predict_1[r] - real[r+1])) / (real[r + 1] * 1.0) ) 
            sum_ = np.sum(sum_)
            sum_ = sum_ / 8.0
            sum_ = math.sqrt(sum_)
            corr = np.corrcoef(Y_pred,Y_test,rowvar=0)[0,1]
            corrcoef.append(corr)
            err.append(sum_)
            if sum_ < best_err:
                best_err = sum_ 
                best_models[rr] = model
                best_models_name[rr] = name[:-4]
            #print name,':\tR²=','%.3f'%corr,'\terr=','%.3f'%sum_
        #print '\n'
        corrcoef_df[rr]=corrcoef
        err_df[rr] = err
        best_errdf[rr] = '%.3f'%best_err
    model_df = pd.DataFrame({'model':best_models,'model_name':best_models_name,'err':best_errdf})
    return err_df,corrcoef_df,model_df

In [4]:
# -*- coding: utf-8 -*- 
def handle_data(data,monthnums):#传入的data为dataframe类型，data需第一列为时间，第二列开始为用户，且行数不得小于30,数据需从1月份开始,monthnums为所需预测的月份
    import pandas as pd
    import numpy as np
    name = data.columns[1:]

    global usernum#用户数量
    global userdata_rows#用户数据行数
    global train_num
    global test_num
    global train_test_num
    global lastmonth#用户所给数据的最后一个年月
    global firstmonth#用户所给数据的第一个年月
    global start_pre_month#预测开始年月
    global end_pre_month#预测结束年月
    global lastmonth_month#用户所给数据的最后一个年月的月份
    global firstmonth_month#用户所给数据的第一个年月的月份
    
    usernum = data.shape[1]-1
    userdata_rows = data.shape[0]
    train_test_num = userdata_rows-15#训练集加上测试集的大小
    train_num = int(train_test_num*0.85)#训练集大小
    test_num = train_test_num-train_num#测试集大小
    lastmonth = data['year'].values[-1]
    firstmonth = data['year'].values[0]
    start_pre_month = data['year'].values[-int(test_num)]
    end_pre_month = int(lastmonth)+monthnums
    lastmonth_month = int(lastmonth[-2]+lastmonth[-1])
    firstmonth_month = int(firstmonth[-2]+firstmonth[-1])
    
    #print '用户所给的最后一个年月数据:',lastmonth
    #print '用户所给的第一个年月数据:',firstmonth
    #print '预测开始年月：',start_pre_month
    #print '预测结束年月:',end_pre_month
    
    
    yuce_month = int(start_pre_month[-2]+start_pre_month[-1])#预测开始月份,从哪里开始截取表格
    if yuce_month>firstmonth_month:
        yuce_year = (int(lastmonth)-int(data['year'].values[yuce_month-firstmonth_month]))/100#表格需要增加的年份
        yuce = data[yuce_month-firstmonth_month:yuce_month-firstmonth_month+test_num+monthnums].copy()#建立表格，后面改数据
    else:
        yuce_year = (int(lastmonth)-int(data['year'].values[yuce_month+(12-firstmonth_month)]))/100
        yuce = data[yuce_month+(12-firstmonth_month):yuce_month+(12-firstmonth_month)+test_num+monthnums].copy()
    yuce.year =yuce.year.astype('int') + yuce_year*100
    
    dfs = {}
    results = {}
    for rr in name:
        train = data[rr]
        tt = np.zeros((train_test_num, 10))

        result = []
        index = 0
        for i in range(15, userdata_rows):
            result.append(train[i])
            tt[index][0] = train[i - 1]
            tt[index][1] = train[i - 2]
            tt[index][2] = train[i - 3]
            tt[index][3] = train[i - 1] + train[i - 2] + train[i - 3]

            tt[index][4] = np.mean(train[i - 3:i])
            tt[index][5] = np.var(train[i - 3:i])

            tt[index][6] = train[i - 12]
            tt[index][7] = train[i - 12 - 1]
            tt[index][8] = train[i - 12 - 2]

            tt[index][9] = (i + 1) % 12
            index += 1
        df = pd.DataFrame(tt,
                          columns=['month_1', 'month_2', 'month_3', 'month_sum', 'month_mean', 'month_var', 'lastmonth_1',
                                   'lastmonth_2', 'lastmonth_3', 'month_date'])

        lastmonth_sum = np.sum(df.iloc[:, 6:9], axis=1)
        df['lastmonth_sum'] = lastmonth_sum

        lastmonth_mean = np.mean(df.iloc[:, 6:9], axis=1)
        df['lastmonth_mean'] = lastmonth_mean
        lastmonth_var = np.var(df.iloc[:, 6:9], axis=1)
        df['lastmonth_var'] = lastmonth_var

        dummy_1 = pd.get_dummies(df['month_date'], prefix='month_date')#转化为one-hot编码
        df.drop(['month_date'], axis=1, inplace=True)
        df = pd.concat([df, dummy_1], axis=1)
        dfs[rr] = df
        results[rr] = result
    return dfs, results,yuce

In [5]:
# -*- coding: utf-8 -*-
def predict_front(dataframe,monthnums,modelnum=0):#dataframe,预测月份数，模型选择,modelnum默认为0，即自动选择用户最优模型
    import matplotlib as plt
    dfs, results, yuce = handle_data(dataframe,monthnums)#获取处理后的数据,训练集，结果集，未修改的预测表格

    names = dataframe.columns[1:]
    err_df,corrcoef_df,model_df = test_model(dfs, results, names)

    user_best_df = pd.DataFrame({'bestmodel':model_df['model_name'],'err':model_df['err']})
    
    if modelnum==0:
        for rr in names:
            names = [rr]
            model_post=user_best_df['bestmodel'][rr]
            locals()['best_models_'+model_post] , locals()['y_predictions_'+model_post] , locals()['err_'+model_post] = eval(model_post)(dfs=dfs,results=results,names=names)
            yuce = predict_behind(best_models=locals()['best_models_'+model_post],y_predictions=locals()['y_predictions_'+model_post],results=results,names=names,yuce=yuce,monthnums=monthnums)
            print '针对用户',rr[-1],':\t',model_post,'最佳模型预测完毕'
            print locals()['err_'+model_post] 
        return yuce
    elif modelnum==1:
        model_post = 'Extrtree'
    elif modelnum==2:
        model_post = 'RF'
    elif modelnum==3:
        model_post = 'Xgboost'
    elif modelnum==4:
        model_post = 'Linear'
    elif modelnum==5:
        model_post = 'GBRT'
    elif modelnum==6:
        model_post = 'Ada'
    elif modelnum==7:
        model_post = 'CART'
    elif modelnum==8:
        model_post = 'EticNet'
    elif modelnum==9:
        model_post = 'Lasso'
    elif modelnum==10:
        model_post = 'Ridge'
        
    print '数据预测中...请稍等...'
    locals()['best_models_'+model_post] , locals()['y_predictions_'+model_post] , locals()['err_'+model_post] = eval(model_post)(dfs=dfs,results=results,names=names)
    yuce = predict_behind(best_models=locals()['best_models_'+model_post],y_predictions=locals()['y_predictions_'+model_post],results=results,names=names,yuce=yuce,monthnums=monthnums)
    print model_post,'模型预测完毕'
    print locals()['err_'+model_post] 
    
    return yuce
    #users_models_err , corrcoef_df ,modle_df= test_model(dfs = dfs, results = results, names = names)#获取最佳模型及模型误差/模型的可决系数
    #print modle_df

In [6]:
# -*- coding: utf-8 -*-
def Ada(dfs,results,names):#返回参数最佳模型，并且返回训练时候的预测结果
    import numpy as np
    import math
    from sklearn.ensemble import AdaBoostRegressor
    from sklearn.externals import joblib
    best_models = {}
    y_predictions = {}
    errs = {}
    num=0.0
    for rr in names:
        user_dfs = dfs[rr]
        
        y_prediction = []
        X_train = user_dfs[0:train_num]
        Y_train = results[rr][0:train_num]

        X_test = user_dfs[train_num:train_test_num]
        Y_test = results[rr][train_num:train_test_num]
        abs_err = 1000000000
        
        model = AdaBoostRegressor()
        model.fit(X_train, Y_train)
        Y_pred = model.predict(X_test)
        sum_  =  ( (Y_test - Y_pred) /(Y_test)) *  (  ((Y_test - Y_pred) ) / (Y_test) )#( abs((predict_1[r] - real[r+1])) / (real[r + 1] * 1.0) ) 
        sum_ = np.sum(sum_)
        sum_ = sum_ / 8.0
        err = math.sqrt(sum_)
        r2 = np.corrcoef(Y_pred,Y_test,rowvar=0)[0,1]
        abs_err = err
        best_model = model
        y_prediction = Y_pred
        
        num = num+1
        #print '参数训练：',num*100/len(names),'%'
        best_models[rr]=best_model
        y_predictions[rr]=y_prediction
        errs[rr]=abs_err
    for rr in names:
        joblib.dump(best_models[rr], "Ada_model/%s_train_model.m"%rr)
        '''
    #使用训练好的模型
        model = joblib.load("Ada_model/%s_train_model.m"%rr)
        model.fit(X_train, Y_train)
        Y_pred = model.predict(X_test)
        y_prediction = Y_pred
        best_models[rr]=model
        y_predictions[rr]=y_prediction
        #print '%s_train_score:'%rr,model.score(X_train,Y_train)
        #print '%s_test_score:'%rr,model.score(X_test,Y_test),'\n'
        global feathername
        feathername = X_train.columns
        #'''
    return best_models,y_predictions,errs

In [7]:
# -*- coding: utf-8 -*-
def CART(dfs,results,names):#返回参数最佳模型，并且返回训练时候的预测结果
    import numpy as np
    import math
    from sklearn.tree import DecisionTreeRegressor
    from sklearn.externals import joblib
    best_models = {}
    y_predictions = {}
    errs = {}
    num=0.0
    for rr in names:
        user_dfs = dfs[rr]
        
        y_prediction = []
        X_train = user_dfs[0:train_num]
        Y_train = results[rr][0:train_num]

        X_test = user_dfs[train_num:train_test_num]
        Y_test = results[rr][train_num:train_test_num]
        abs_err = 1000000000
        
        model = DecisionTreeRegressor()
        model.fit(X_train, Y_train)
        Y_pred = model.predict(X_test)
        sum_  =  ( (Y_test - Y_pred) /(Y_test)) *  (  ((Y_test - Y_pred) ) / (Y_test) )#( abs((predict_1[r] - real[r+1])) / (real[r + 1] * 1.0) ) 
        sum_ = np.sum(sum_)
        sum_ = sum_ / 8.0
        err = math.sqrt(sum_)
        r2 = np.corrcoef(Y_pred,Y_test,rowvar=0)[0,1]
        abs_err = err
        best_model = model
        y_prediction = Y_pred
        
        num = num+1
        #print '参数训练：',num*100/len(names),'%'
        best_models[rr]=best_model
        y_predictions[rr]=y_prediction
        errs[rr]=abs_err
    for rr in names:
        joblib.dump(best_models[rr], "CART_model/%s_train_model.m"%rr)
        '''
    #使用训练好的模型
        model = joblib.load("CART_model/%s_train_model.m"%rr)
        model.fit(X_train, Y_train)
        Y_pred = model.predict(X_test)
        y_prediction = Y_pred
        best_models[rr]=model
        y_predictions[rr]=y_prediction
        #print '%s_train_score:'%rr,model.score(X_train,Y_train)
        #print '%s_test_score:'%rr,model.score(X_test,Y_test),'\n'
        global feathername
        feathername = X_train.columns
        #'''
    return best_models,y_predictions,errs

In [8]:
# -*- coding: utf-8 -*-
def EticNet(dfs,results,names):#返回参数最佳模型，并且返回训练时候的预测结果
    import numpy as np
    import math
    from sklearn.linear_model import ElasticNet
    from sklearn.externals import joblib
    best_models = {}
    y_predictions = {}
    errs = {}
    num=0.0
    for rr in names:
        user_dfs = dfs[rr]
        
        y_prediction = []
        X_train = user_dfs[0:train_num]
        Y_train = results[rr][0:train_num]

        X_test = user_dfs[train_num:train_test_num]
        Y_test = results[rr][train_num:train_test_num]
        abs_err = 1000000000
        
        model = ElasticNet()
        model.fit(X_train, Y_train)
        Y_pred = model.predict(X_test)
        sum_  =  ( (Y_test - Y_pred) /(Y_test)) *  (  ((Y_test - Y_pred) ) / (Y_test) )#( abs((predict_1[r] - real[r+1])) / (real[r + 1] * 1.0) ) 
        sum_ = np.sum(sum_)
        sum_ = sum_ / 8.0
        err = math.sqrt(sum_)
        r2 = np.corrcoef(Y_pred,Y_test,rowvar=0)[0,1]
        abs_err = err
        best_model = model
        y_prediction = Y_pred
        
        num = num+1
        #print '参数训练：',num*100/len(names),'%'
        best_models[rr]=best_model
        y_predictions[rr]=y_prediction
        errs[rr]=abs_err
    for rr in names:
        joblib.dump(best_models[rr], "ElasticNet_model/%s_train_model.m"%rr)
        '''
    #使用训练好的模型
        model = joblib.load("ElasticNet_model/%s_train_model.m"%rr)
        model.fit(X_train, Y_train)
        Y_pred = model.predict(X_test)
        y_prediction = Y_pred
        best_models[rr]=model
        y_predictions[rr]=y_prediction
        #print '%s_train_score:'%rr,model.score(X_train,Y_train)
        #print '%s_test_score:'%rr,model.score(X_test,Y_test),'\n'
        global feathername
        feathername = X_train.columns
        #'''
    return best_models,y_predictions,errs

In [9]:
# -*- coding: utf-8 -*-
def Lasso(dfs,results,names):#返回参数最佳模型，并且返回训练时候的预测结果
    import numpy as np
    import math
    from sklearn.linear_model import Lasso
    from sklearn.externals import joblib
    best_models = {}
    y_predictions = {}
    errs = {}
    num=0.0
    for rr in names:
        user_dfs = dfs[rr]
        
        y_prediction = []
        X_train = user_dfs[0:train_num]
        Y_train = results[rr][0:train_num]

        X_test = user_dfs[train_num:train_test_num]
        Y_test = results[rr][train_num:train_test_num]
        abs_err = 1000000000
        
        model = Lasso()
        model.fit(X_train, Y_train)
        Y_pred = model.predict(X_test)
        sum_  =  ( (Y_test - Y_pred) /(Y_test)) *  (  ((Y_test - Y_pred) ) / (Y_test) )#( abs((predict_1[r] - real[r+1])) / (real[r + 1] * 1.0) ) 
        sum_ = np.sum(sum_)
        sum_ = sum_ / 8.0
        err = math.sqrt(sum_)
        r2 = np.corrcoef(Y_pred,Y_test,rowvar=0)[0,1]
        abs_err = err
        best_model = model
        y_prediction = Y_pred
        
        num = num+1
        #print '参数训练：',num*100/len(names),'%'
        best_models[rr]=best_model
        y_predictions[rr]=y_prediction
        errs[rr]=abs_err
    for rr in names:
        joblib.dump(best_models[rr], "Lasso_model/%s_train_model.m"%rr)
        '''
    #使用训练好的模型
        model = joblib.load("Lasso_model/%s_train_model.m"%rr)
        model.fit(X_train, Y_train)
        Y_pred = model.predict(X_test)
        y_prediction = Y_pred
        best_models[rr]=model
        y_predictions[rr]=y_prediction
        #print '%s_train_score:'%rr,model.score(X_train,Y_train)
        #print '%s_test_score:'%rr,model.score(X_test,Y_test),'\n'
        global feathername
        feathername = X_train.columns
        #'''
    return best_models,y_predictions,errs

In [10]:
# -*- coding: utf-8 -*-
def Ridge(dfs,results,names):#返回参数最佳模型，并且返回训练时候的预测结果
    import numpy as np
    import math
    from sklearn.linear_model import Ridge
    from sklearn.externals import joblib
    best_models = {}
    y_predictions = {}
    errs = {}
    num=0.0
    for rr in names:
        user_dfs = dfs[rr]
        
        y_prediction = []
        X_train = user_dfs[0:train_num]
        Y_train = results[rr][0:train_num]

        X_test = user_dfs[train_num:train_test_num]
        Y_test = results[rr][train_num:train_test_num]
        abs_err = 1000000000
        
        model = Ridge()
        model.fit(X_train, Y_train)
        Y_pred = model.predict(X_test)
        sum_  =  ( (Y_test - Y_pred) /(Y_test)) *  (  ((Y_test - Y_pred) ) / (Y_test) )#( abs((predict_1[r] - real[r+1])) / (real[r + 1] * 1.0) ) 
        sum_ = np.sum(sum_)
        sum_ = sum_ / 8.0
        err = math.sqrt(sum_)
        r2 = np.corrcoef(Y_pred,Y_test,rowvar=0)[0,1]
        abs_err = err
        best_model = model
        y_prediction = Y_pred
        
        num = num+1
        #print '参数训练：',num*100/len(names),'%'
        best_models[rr]=best_model
        y_predictions[rr]=y_prediction
        errs[rr]=abs_err
    for rr in names:
        joblib.dump(best_models[rr], "Ridge_model/%s_train_model.m"%rr)
        '''
    #使用训练好的模型
        model = joblib.load("Ridge_model/%s_train_model.m"%rr)
        model.fit(X_train, Y_train)
        Y_pred = model.predict(X_test)
        y_prediction = Y_pred
        best_models[rr]=model
        y_predictions[rr]=y_prediction
        #print '%s_train_score:'%rr,model.score(X_train,Y_train)
        #print '%s_test_score:'%rr,model.score(X_test,Y_test),'\n'
        global feathername
        feathername = X_train.columns
        #'''
    return best_models,y_predictions,errs

In [11]:
# -*- coding: utf-8 -*-
def GBRT(dfs,results,names):#返回参数最佳模型，并且返回训练时候的预测结果
    import numpy as np
    import math
    from sklearn.ensemble import GradientBoostingRegressor
    from sklearn.externals import joblib
    best_models = {}
    y_predictions = {}
    errs = {}
    num=0.0
    for rr in names:
        user_dfs = dfs[rr]
        
        y_prediction = []
        X_train = user_dfs[0:train_num]
        Y_train = results[rr][0:train_num]

        X_test = user_dfs[train_num:train_test_num]
        Y_test = results[rr][train_num:train_test_num]
        abs_err = 1000000000
        
        model = GradientBoostingRegressor()
        model.fit(X_train, Y_train)
        Y_pred = model.predict(X_test)
        sum_  =  ( (Y_test - Y_pred) /(Y_test)) *  (  ((Y_test - Y_pred) ) / (Y_test) )#( abs((predict_1[r] - real[r+1])) / (real[r + 1] * 1.0) ) 
        sum_ = np.sum(sum_)
        sum_ = sum_ / 8.0
        err = math.sqrt(sum_)
        r2 = np.corrcoef(Y_pred,Y_test,rowvar=0)[0,1]
        abs_err = err
        best_model = model
        y_prediction = Y_pred
        
        num = num+1
        #print '参数训练：',num*100/len(names),'%'
        best_models[rr]=best_model
        y_predictions[rr]=y_prediction
        errs[rr]=abs_err
    for rr in names:
        joblib.dump(best_models[rr], "GBRT_model/%s_train_model.m"%rr)
        '''
    #使用训练好的模型
        model = joblib.load("GBRT_model/%s_train_model.m"%rr)
        model.fit(X_train, Y_train)
        Y_pred = model.predict(X_test)
        y_prediction = Y_pred
        best_models[rr]=model
        y_predictions[rr]=y_prediction
        #print '%s_train_score:'%rr,model.score(X_train,Y_train)
        #print '%s_test_score:'%rr,model.score(X_test,Y_test),'\n'
        global feathername
        feathername = X_train.columns
        #'''
    return best_models,y_predictions,errs

In [12]:
# -*- coding: utf-8 -*-
def Linear(dfs,results,names):#返回参数最佳模型，并且返回训练时候的预测结果
    import numpy as np
    import math
    from sklearn.linear_model import LinearRegression
    from sklearn.externals import joblib
    best_models = {}
    y_predictions = {}
    errs = {}
    num=0.0
    for rr in names:
        user_dfs = dfs[rr]
        
        y_prediction = []
        X_train = user_dfs[0:train_num]
        Y_train = results[rr][0:train_num]

        X_test = user_dfs[train_num:train_test_num]
        Y_test = results[rr][train_num:train_test_num]
        abs_err = 1000000000
        
        model = LinearRegression()
        model.fit(X_train, Y_train)
        Y_pred = model.predict(X_test)
        sum_  =  ( (Y_test - Y_pred) /(Y_test)) *  (  ((Y_test - Y_pred) ) / (Y_test) )#( abs((predict_1[r] - real[r+1])) / (real[r + 1] * 1.0) ) 
        sum_ = np.sum(sum_)
        sum_ = sum_ / 8.0
        err = math.sqrt(sum_)
        r2 = np.corrcoef(Y_pred,Y_test,rowvar=0)[0,1]
        abs_err = err
        best_model = model
        y_prediction = Y_pred
        
        num = num+1
        #print '参数训练：',num*100/len(names),'%'
        best_models[rr]=best_model
        y_predictions[rr]=y_prediction
        errs[rr]=abs_err
    for rr in names:
        joblib.dump(best_models[rr], "Linear_model/%s_train_model.m"%rr)
        '''
    #使用训练好的模型
        model = joblib.load("Linear_model/%s_train_model.m"%rr)
        model.fit(X_train, Y_train)
        Y_pred = model.predict(X_test)
        y_prediction = Y_pred
        best_models[rr]=model
        y_predictions[rr]=y_prediction
        #print '%s_train_score:'%rr,model.score(X_train,Y_train)
        #print '%s_test_score:'%rr,model.score(X_test,Y_test),'\n'
        global feathername
        feathername = X_train.columns
        #'''
    return best_models,y_predictions,errs

In [13]:
# -*- coding: utf-8 -*-
def RF(dfs,results,names):#返回参数最佳模型，并且返回训练时候的预测结果
    import numpy as np
    import math
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.externals import joblib
    best_models = {}
    y_predictions = {}
    errs = {}
    num=0.0
    for rr in names:
        user_dfs = dfs[rr]
        
        y_prediction = []
        X_train = user_dfs[0:train_num]
        Y_train = results[rr][0:train_num]

        X_test = user_dfs[train_num:train_test_num]
        Y_test = results[rr][train_num:train_test_num]
        abs_err = 1000000000

        #'''
        #learn_rate = 0
        feature_rate = 0
        nestimator = 0
        max_depth = 0 
        samples_leaf = 0
        model_yuce = 0
        for n_round in [150,250,400]:
            for depth in [5,6]:
                for leaf in [1,2]: 
                    model = RandomForestRegressor(criterion='mse', max_depth = depth,
                          max_features='auto',  min_samples_leaf=leaf,
                          min_samples_split=2, n_estimators=  n_round, random_state=1126)

                    model.fit(X_train, Y_train)
                    #y_trainpre = model.predict(X_train)
                    Y_pred = model.predict(X_test)
                        #msg = "%s: " % (name)
                        #print(rr)
                        #err = metrics.mean_absolute_error(y_test, y_pred)
                    sum_  =  ( (Y_test - Y_pred) /(Y_test)) *  (  ((Y_test - Y_pred) ) / (Y_test) )#( abs((predict_1[r] - real[r+1])) / (real[r + 1] * 1.0) ) 
                    sum_ = np.sum(sum_)
                    sum_ = sum_ / 8.0
                    err = math.sqrt(sum_)
                    #err_p.append(sum)
                    #print (msg)," \t%.4g" % sum_#metrics.mean_absolute_error(y_test, y_pred)  
                    if err < abs_err:
                        r2 = np.corrcoef(Y_pred,Y_test,rowvar=0)[0,1]
                        abs_err = err
                        best_model = model
                        y_prediction = Y_pred
        num = num+1
        #print '参数训练：',num*100/len(names),'%'
        best_models[rr]=best_model
        y_predictions[rr]=y_prediction
        errs[rr]=abs_err
    for rr in names:
        joblib.dump(best_models[rr], "RF_model/%s_train_model.m"%rr)
    '''
    #使用训练好的模型
        model = joblib.load("RF_model/%s_train_model.m"%rr)
        model.fit(X_train, Y_train)
        Y_pred = model.predict(X_test)
        y_prediction = Y_pred
        best_models[rr]=model
        y_predictions[rr]=y_prediction
        #print '%s_train_score:'%rr,model.score(X_train,Y_train)
        #print '%s_test_score:'%rr,model.score(X_test,Y_test),'\n'
        global feathername
        feathername = X_train.columns
    '''     
    return best_models,y_predictions,errs

In [14]:
# -*- coding: utf-8 -*-
def Extrtree(dfs,results,names):#返回参数最佳模型，并且返回训练时候的预测结果
    import numpy as np
    import math
    from sklearn.ensemble import ExtraTreesRegressor
    from sklearn.externals import joblib
    best_models = {}
    y_predictions = {}
    errs = {}
    num=0.0
    for rr in names:
        user_dfs = dfs[rr]
        
        y_prediction = []
        X_train = user_dfs[0:train_num]
        Y_train = results[rr][0:train_num]

        X_test = user_dfs[train_num:train_test_num]
        Y_test = results[rr][train_num:train_test_num]
        abs_err = 1000000000

        ExtraTreesRegressor(bootstrap=False, criterion='mse', max_depth=None,
              max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
              verbose=0, warm_start=False)
        #'''
        #learn_rate = 0
        feature_rate = 0
        nestimator = 0
        max_depth = 0 
        samples_leaf = 0
        model_yuce = 0
        for n_round in [150,250,400]:
            for feature in [0.9,1.0]:
                for depth in [5,6]:
                    for leaf in [1,2]: 
                        model = ExtraTreesRegressor(criterion='mse', max_depth = depth,
                              max_features=feature,  min_samples_leaf=leaf,
                              min_samples_split=2, n_estimators=  n_round, random_state=1126)

                        model.fit(X_train, Y_train)
                        #y_trainpre = model.predict(X_train)
                        Y_pred = model.predict(X_test)
                            #msg = "%s: " % (name)
                            #print(rr)
                            #err = metrics.mean_absolute_error(y_test, y_pred)
                        sum_  =  ( (Y_test - Y_pred) /(Y_test)) *  (  ((Y_test - Y_pred) ) / (Y_test) )#( abs((predict_1[r] - real[r+1])) / (real[r + 1] * 1.0) ) 
                        sum_ = np.sum(sum_)
                        sum_ = sum_ / 8.0
                        err = math.sqrt(sum_)
                        #err_p.append(sum)
                        #print (msg)," \t%.4g" % sum_#metrics.mean_absolute_error(y_test, y_pred)  
                        if err < abs_err:
                            r2 = np.corrcoef(Y_pred,Y_test,rowvar=0)[0,1]
                            abs_err = err
                            best_model = model
                            y_prediction = Y_pred
        num = num+1
        #print '参数训练：',num*100/len(names),'%'
        best_models[rr]=best_model
        y_predictions[rr]=y_prediction
        errs[rr]=abs_err
    for rr in names:
        joblib.dump(best_models[rr], "Extr_model/%s_train_model.m"%rr)
    '''
    #使用训练好的模型
        model = joblib.load("Extr_model/%s_train_model.m"%rr)
        model.fit(X_train, Y_train)
        Y_pred = model.predict(X_test)
        y_prediction = Y_pred
        best_models[rr]=model
        y_predictions[rr]=y_prediction
        #print '%s_train_score:'%rr,model.score(X_train,Y_train)
        #print '%s_test_score:'%rr,model.score(X_test,Y_test),'\n'
        global feathername
        feathername = X_train.columns
    '''
    return best_models,y_predictions,errs

In [15]:
# -*- coding: utf-8 -*-
def Xgboost(dfs,results,names):
    from xgboost import XGBRegressor
    import numpy as np
    import math
    from sklearn.externals import joblib
    best_models = {}
    y_predictions = {}
    errs = {}
    num=0.0
    for rr in names:
        user_dfs = dfs[rr]
        
        y_prediction = []
        X_train = user_dfs[0:train_num]
        Y_train = results[rr][0:train_num]

        X_test = user_dfs[train_num:train_test_num]
        Y_test = results[rr][train_num:train_test_num]
        abs_err = 1000000000
        #需要调用保存好的模型则从这里开始注释掉
        #'''
        #learn_rate = 0
        feature_rate = 0
        nestimator = 0
        max_depth = 0 
        samples_leaf = 0
        model_yuce = 0
        for lr in [0.1,0.05,0.01,0.005]:
            for n_estimator in [300,500,800,1000]:
                for max_depth in [4,5,6]:
                    for min_child_weight in [1,2,3,4,5,6]:
                                        model = XGBRegressor(learning_rate =lr,n_estimators=n_estimator ,max_depth=max_depth,min_child_weight=min_child_weight,gamma=1,subsample=1.0,
                                        colsample_bytree=1.0,reg_alpha=1.0,objective= 'reg:linear',nthread=-1,scale_pos_weight=1,seed=1126)
                                        eval_set = [(X_test, Y_test)]
                                        model.fit(X_train, Y_train, eval_metric=['rmse'], eval_set=eval_set,verbose=False,early_stopping_rounds = 30)
                                        #y_trainpre = model.predict(X_train)
                                        Y_pred = model.predict(X_test)
                                                #msg = "%s: " % (name)
                                                #print(rr)
                                                #err = metrics.mean_absolute_error(y_test, y_pred)
                                        sum_  =  ( (Y_test - Y_pred) /(Y_test)) *  (  ((Y_test - Y_pred) ) / (Y_test) )#( abs((predict_1[r] - real[r+1])) / (real[r + 1] * 1.0) ) 
                                        sum_ = np.sum(sum_)
                                        sum_ = sum_ / 8.0
                                        err = math.sqrt(sum_)
                                        #err_p.append(sum)
                                        #print (msg)," \t%.4g" % sum_#metrics.mean_absolute_error(y_test, y_pred)  
                                        if err < abs_err:
                                                r2 = np.corrcoef(Y_pred,Y_test,rowvar=0)[0,1]
                                                abs_err = err
                                                best_model = model
                                                y_prediction = Y_pred
        num = num+1
        #print '参数训练：',num*100/len(names),'%'
        best_models[rr]=best_model
        y_predictions[rr]=y_prediction
        errs[rr]=abs_err
    for rr in names:
        joblib.dump(best_models[rr], "Xgb_model/%s_train_model.m"%rr)#保存训练模型
    '''
    #读取使用训练好的模型，为了方便测试而已
        model = joblib.load("Xgb_model/%s_train_model.m"%rr)
        model.fit(X_train, Y_train)

        Y_pred = model.predict(X_test)
        y_prediction = Y_pred
        best_models[rr]=model
        y_predictions[rr]=y_prediction
    
        #print '%s_train_score:'%rr,model.score(X_train,Y_train)
        #print '%s_test_score:'%rr,model.score(X_test,Y_test),'\n'
    '''
    return best_models,y_predictions,errs

In [16]:
# -*- coding: utf-8 -*-
def predict_behind(best_models,y_predictions,results,names,yuce,monthnums):#最佳模型，训练时候预测的结果，用户数据,用户名字,待填入的表格,预测的月份数
    import numpy as np
    import pandas as pd
    for rr in names:
        model = best_models[rr]
        y_prediction = y_predictions[rr].tolist()
        result = results[rr]
        for d in range(lastmonth_month+1,lastmonth_month+monthnums+1):
            val = []
            val.append(result[-1])
            val.append(result[-2])
            val.append(result[-3])

            dd1 = np.sum(result[-3:])
            dd2 = np.mean(result[-3:])
            dd3 = np.var(result[-3:])

            val.append(dd1)
            val.append(dd2)
            val.append(dd3)

            val.append(result[-13])
            val.append(result[-14])
            val.append(result[-15])

            df_1 = pd.DataFrame(np.array(val).reshape(1,9),columns=['month_1','month_2','month_3','month_sum','month_mean','month_var','lastmonth_1','lastmonth_2','lastmonth_3'])

            lastmonth_sum = np.sum (df_1.iloc[:,6:9],axis = 1 )
            df_1['lastmonth_sum'] = lastmonth_sum

            lastmonth_mean = np.mean(df_1.iloc[:,6:9],axis = 1 )
            df_1['lastmonth_mean'] = lastmonth_mean

            lastmonth_var = np.var(df_1.iloc[:,6:9],axis = 1 )
            df_1['lastmonth_var']  =  lastmonth_var
            strs = 'month_date_'
            for j in range(1,13):
                d = d%12
                if d==0:
                    d=12
                col= strs + str(j-1)+'.0'
                if j == d:
                    df_1[col] = 1
                else:
                    df_1[col] = 0
            #print df_1
            pr =model.predict(df_1)#.values)
            y_prediction.append(pr[0])

            result.append(pr[0])
        yuce[rr] = y_prediction#此处修改表格
    return yuce

In [18]:
# -*- coding: utf-8 -*-
import pandas as pd
import matplotlib as plt
from sklearn.externals import joblib
data = pd.read_csv('user.csv',sep=',',parse_dates=[0])
data2 = pd.DataFrame(data['year'][11:51])
data2['UserB']=data['UserB'][11:51]
data2['UserC']=data['UserC'][11:51]
data2['UserD']=data['UserD'][11:51]
data2['UserF']=data['UserF'][11:51]
data2 = data2.reset_index(drop=True)#重置索引

#print data2
yuce = predict_front(data,16,0)
yuce

针对用户 l :	Xgboost 最佳模型预测完毕
{'total': 0.06737014930347388}
针对用户 A :	Extrtree 最佳模型预测完毕
{'UserA': 0.06048218937123648}
针对用户 B :	Xgboost 最佳模型预测完毕
{'UserB': 0.0667111196852653}
针对用户 C :	Extrtree 最佳模型预测完毕
{'UserC': 0.09232319531917232}
针对用户 D :	GBRT 最佳模型预测完毕
{'UserD': 0.07719744214418763}
针对用户 F :	Extrtree 最佳模型预测完毕
{'UserF': 0.08164922363673457}
针对用户 G :	GBRT 最佳模型预测完毕
{'UserG': 0.0663972107310566}
针对用户 H :	GBRT 最佳模型预测完毕
{'UserH': 0.05643088544705206}
针对用户 I :	Extrtree 最佳模型预测完毕
{'UserI': 0.04958745194960609}
针对用户 J :	Extrtree 最佳模型预测完毕
{'UserJ': 0.050872992342386486}
针对用户 K :	Ada 最佳模型预测完毕
{'UserK': 0.13243765296523816}
针对用户 L :	Lasso 最佳模型预测完毕
{'UserL': 0.1276831646307825}
针对用户 M :	Ridge 最佳模型预测完毕
{'UserM': 0.07156455352935115}
针对用户 N :	Extrtree 最佳模型预测完毕
{'UserN': 0.0646719680466625}
针对用户 O :	Extrtree 最佳模型预测完毕
{'UserO': 0.1068069611761291}
针对用户 P :	Ada 最佳模型预测完毕
{'UserP': 0.12266658254508242}
针对用户 Q :	GBRT 最佳模型预测完毕
{'UserQ': 0.17906108897486028}
针对用户 R :	EticNet 最佳模型预测完毕
{'UserR': 0.07386824097124

,year,total,UserA,UserB,UserC,UserD,UserF,UserG,UserH,UserI,UserJ,UserK,UserL,UserM,UserN,UserO,UserP,UserQ,UserR,UserS,UserT
12,201601,15979988.0,3.051912e+06,1.150270e+06,191407.212980,1.635866e+06,224899.004958,144510.265020,1.354265e+06,767313.600287,208998.650259,28120.216111,813956.208213,48603.725009,194801.128969,72908.245235,3.830154e+06,403675.256263,188795.379502,198504.559395,940625.994248
13,201602,12985894.0,2.160328e+06,9.711174e+05,142933.137286,1.451552e+06,205994.273438,124587.141992,1.036860e+06,630030.684465,206616.521922,29587.394286,470807.462551,49742.271828,174992.341804,71490.610076,3.281438e+06,319720.075075,174927.402561,206111.109986,621598.852339
14,201603,6747719.0,1.079271e+06,4.359702e+05,81287.580000,8.304113e+05,169518.186074,72780.203037,5.389180e+05,405086.201433,103787.613333,19935.010000,329249.123418,43921.284619,110487.748400,40460.465333,2.156520e+06,246375.115785,146115.943498,94828.078891,288835.328287
15,201604,12470456.0,2.587162e+06,8.811551e+05,167055.656106,1.397489e+06,181805.081550,106987.959970,1.093118e+06,642859.017173,189599.042703,27642.315556,736830.295549,50539.230316,167015.575945,68152.608878,3.124189e+06,374539.051437,182918.235495,150024.121285,726887.543609
16,201605,13729152.0,2.668969e+06,1.002284e+06,168778.101530,1.416435e+06,178521.772801,126351.751933,1.121195e+06,640303.772148,192213.432478,28675.035714,832725.568996,52451.285021,170696.312695,68592.871297,3.356582e+06,334495.544922,174278.396589,219997.090698,785295.794359
17,201606,14605678.0,2.712199e+06,1.015808e+06,168848.499658,1.439867e+06,214580.821299,120987.925159,1.156117e+06,708384.090289,204265.890551,29824.907647,782631.357947,58461.110606,188823.454416,71820.619524,3.504151e+06,326837.930569,179313.639990,200629.457124,832223.924543
18,201607,13856424.0,2.701909e+06,1.001022e+06,172089.040289,1.451293e+06,208270.092742,119732.567873,1.137532e+06,701124.454440,208756.431519,29515.158889,744941.197094,60076.790325,194088.652491,73530.375645,3.605277e+06,329202.105729,190585.387329,169548.253612,810235.121914
19,201608,13771118.0,2.723870e+06,9.727777e+05,172707.556945,1.485771e+06,211907.646541,121614.216462,1.171605e+06,716192.260008,213377.912877,29757.286364,762963.347064,60321.516217,199392.843007,73684.593561,3.776873e+06,342741.597287,200248.531597,163063.209288,814764.306365
20,201609,13537708.0,2.728035e+06,9.660276e+05,177615.523482,1.438753e+06,210133.526305,117367.219262,1.171501e+06,715153.761908,212169.160802,29757.286364,782078.676518,62157.439482,203307.719675,73697.680481,3.774499e+06,302894.097812,214713.623112,167598.547413,814764.306365
21,201610,13431980.0,2.715863e+06,9.646596e+05,181845.041240,1.421970e+06,206099.656885,122723.115929,1.147484e+06,712873.460088,212868.186037,29824.907647,748791.336893,64968.879911,208429.610494,73440.611002,3.602492e+06,319687.359067,214074.475602,176970.952879,822048.386548


In [56]:
def draw(treemodel,feature_names):
    from IPython.display import Image,display
    from sklearn import tree
    import pydotplus 
    dot_data = tree.export_graphviz(treemodel, out_file=None, 
                         feature_names=feature_names,  
                         filled=True, rounded=True,  
                         special_characters=True)
    graph = pydotplus.graph_from_dot_data(dot_data)
    graph.write_pdf('RF_tree3.pdf')
    display(Image(graph.create_png()))

In [33]:
yuce.to_csv('electpre_XGB_model.csv',index = False)

In [19]:
names = data.columns[1:]
for rr in names[1:2]:
    print rr
    '''
    model = joblib.load("Extr_model/%s_train_model.m"%rr)
    mytree = model.estimators_[5]
    draw(mytree,feathername)
    
    print feathername
    model2 = joblib.load("RF_model/%s_train_model.m"%rr)
    mytree2 = model2.estimators_[37]
    draw(mytree2,feathername)
    '''

UserA


In [141]:
import pandas as pd 
test = pd.read_csv('originalData.csv',sep=',',parse_dates=[0]) 
test2 = test.drop(test.columns[[0,1]],axis=1) 

my = test2.apply(lambda x: x.mean())

my_dict = {}

for i in my.index:
    bit = len(str(int(my[i])))
    if my_dict.has_key(bit):
        my_dict[bit].append(i)
    else:
        my_dict[bit] = [i]
print my_dict

{5: ['UserK', 'UserM', 'UserO'], 6: ['UserB', 'UserC', 'UserF', 'UserG', 'UserI', 'UserJ', 'UserL', 'UserN', 'UserQ', 'UserR', 'UserS', 'UserT'], 7: ['UserA', 'UserD', 'UserH', 'UserP']}


In [125]:
dic = {1:[2]}
print dic
dic[1].append(3)
print dic

{1: [2]}
{1: [2, 3]}


In [ ]:
UserA : 8
UserB : 8
UserC : 7
UserD : 8
UserF : 7
UserG : 7
UserH : 8
UserI : 8
UserJ : 7
UserK : 6
UserL : 8
UserM : 7
UserN : 7
UserO : 7
UserP : 8
UserQ : 7
UserR : 7
UserS : 7
UserT : 8